In [1]:
import pandas as pd
import numpy as np
import re

import csv

import json

from scipy.stats import trim_mean  
from scipy import stats
from scipy.stats import sem

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import spacy

In [2]:
fuzz.partial_ratio("this is a test", "this is a test!")

100

In [3]:
route = '../../../data/'

In [4]:
route_2 = '../datasets/'

# Importación de DF producto limpio

In [5]:
df = pd.read_csv(route + 'datamarket_clean_unique.csv')
#df.head()

# Importación DF listado de recetas

In [6]:
df_recetas = pd.read_csv(route + 'df_recetas.csv', index_col=0)
#df_recetas.info()

In [7]:
# Necesario cambiar el formato de la columna puesto que DF lo guarda como un string y no se puede luego iterar.

df_recetas["ingredientes"] = df_recetas["ingredientes"].apply(eval)

In [10]:
df_recetas["ingredientes"][9]

['200 g de zanahoria troceada',
 '2 dientes de ajo',
 '1 - 2 ramitas de perejil fresco (sólo las hojas)',
 '50 g de pan rallado',
 '2 huevos',
 '50 g de leche',
 '1 cucharadita de sal',
 '1 pellizco de nuez moscada molida',
 '1 pellizco de canela molida',
 '1 pellizco de pimienta negra molida',
 '500 g de carne picada mixta (250 g de ternera y 250 g de cerdo)',
 '50 g de piñones (opcional)',
 '20 g de aceite de oliva virgen extra (para engrasar la bandeja)',
 'harina (para rebozar)',
 '250 g de cebolla cortada en cuartos',
 '2 dientes de ajo',
 '50 g de aceite de oliva virgen extra',
 '1 cucharada de tomate concentrado',
 '100 g de vino blanco',
 '300 g de agua',
 '1 pellizco de sal',
 '1 pellizco nuez moscada molida',
 '1 pellizco de pimienta negra molida',
 '1 cucharada de extracto de buey',
 '1 pellizco de clavo molido',
 '1 cucharada de perejil fresco picado']

### Listas intermedias

In [11]:
# Opening JSON file
with open(route + 'data_mercadona.json') as json_file:
    a_mercadona = json.load(json_file)
#a_mercadona

In [12]:
with open(route + 'data_dia.json') as json_file:
    a_dia = json.load(json_file)
#a_dia

{'b26d51cb5be3aff71a71f87c8908b846': 'lenteja',
 'dada7a323f98cbb8abd3b7a65f11da13': 'pimiento verde',
 '3b118605318ccdf76e4db430ef53f94c': 'pimiento rojo',
 '6d854922c4f8a5a5740991f3e7108da0': 'zanahoria',
 '06690951566cb9affcb7b886db944225': 'cebolletar',
 '9dff07cb2f5875147d574f3cd2eb52d7': 'cebolla',
 'ff4bec68a3103927d5568c80482a6c54': 'aceite olivo virgen extra',
 '9e6edc9fb8588e953a8177db9c5fa2c5': 'aceite olivo',
 'e453450e28331ea2f9d7b32f423f0ae0': 'sal',
 '213ebf1ba8e25da18f50e6770a6fb2bb': 'pimentón',
 '068082a32f9ba83d41c4db0df5087004': 'pastilla de caldo de pollo',
 '616df75e8d66ded901d003e282c3e062': 'caldo de pollo',
 '26b1a3be138796cd829876975f1d1b01': 'chorizo',
 '640a8ef50612a09e27d47a75f8ac10f4': 'jamón curado dado',
 'c2790645ef08d7603f9acb25ecc96f70': 'agua',
 '7a9393a2992597b111da5b14fa16c1c5': 'diente ajo',
 'adbd6277e8d208701b9ec82ec0e557cd': 'tomate troceado conserva',
 '2744830b16f0aa447e2216e9b6dca13b': 'pimiento piquillo conserva',
 '2f1c6f49d5b2a6ed34cd8031

In [13]:
with open(route + 'data_carrefour.json') as json_file:
    a_carrefour = json.load(json_file)
#a_carrefour

In [14]:
tablas_intermedias = [a_mercadona, a_dia, a_carrefour]

In [15]:
nombres_supermercados = ['mercadona', 'dia', 'carrefour']

In [16]:
nlp = spacy.load('es_core_news_sm')

In [17]:
def normalize_lemma_space(text):
    doc = nlp(text)
    words = [t.orth_ for t in doc if not t.is_punct | t.is_stop]
    lexical_tokens = [t.lower() for t in words if len(t) > 2 and t.isalpha()]
    cadena = " ".join(lexical_tokens)
    doc2 = nlp(cadena)
    lemmas = [tok.lemma_.lower() for tok in doc2 if tok.pos_ != 'PRON']
    lemmas_cadena = " ".join(lemmas)
    return lemmas_cadena

In [18]:
def get_key_generico(ingrediente, lista_supermercado):
    
    # list out keys and values separately
    key_list = list(lista_supermercado.keys())
    val_list = list(lista_supermercado.values())
    
    # lematizamos ingrediente primero
    ingrediente_lemma = normalize_lemma_space(ingrediente)
    
    # hacemos fuzzybuzzy sobre la lista intermedia
    ratios = process.extractOne(ingrediente_lemma, lista_supermercado)
    
    if int(ratios[1]) > 70: # can play with this number 
        position = val_list.index(ratios[0])
        return [key_list[position], ratios[0], ratios[1]]
 
    return ['No product_ID', 'No product Match', 'No product Match']

In [19]:
def precio_gramaje(ingrediente, df_clean, prod_id):                
    gr = word_tokenize(ingrediente, language='spanish')
    bolsa_tinta = 0.025
    cucharada = 0.075
    cucharadita = 0.005
    diente = 0.002
    hebra = 0.0005
    pastilla = 0.005
    pellizco = 0.001
    placa_lasaña = 0.01
    ramita_perejil = 0.001
    if gr[1] == 'g':
        x = (int(gr[0])/1000) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
    elif gr[1] == 'kg':
        x = int(gr[0]) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
    elif gr[1] == 'l':
        x = int(gr[0]) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
    elif gr[1] == 'hebra':
        x = (int(gr[0]) * hebra) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
    elif gr[1] == 'hebras':
        x = (int(gr[0]) * hebra) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
    elif gr[1] == 'bolsa':
        x = (int(gr[0]) * bolsa_tinta) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
    elif gr[1] == 'bolsas':
        x = (int(gr[0]) * bolsa_tinta) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
    elif gr[1] == 'placa':
        x = (int(gr[0]) * placa_lasaña) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
    elif gr[1] == 'placas':
        x = (int(gr[0]) * placa_lasaña) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
    elif gr[1] == 'cucharadita':
        if gr[0] == '½':
            x = (0.5*cucharadita) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        else:
            x = (int(gr[0])*cucharadita) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
    elif gr[1] == 'cucharaditas':
        if gr[0] == '½':
            x = (0.5*cucharadita) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        else:
            x = (int(gr[0])*cucharadita) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
    elif gr[1] == 'cucharada':
        if gr[0] == '½':
            x = (0.5*cucharada) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        else:
            x = (int(gr[0])*cucharada) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
    elif gr[1] == 'cucharadas':
        if gr[0] == '½':
            x = (0.5*cucharada) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        else:
            x = (int(gr[0])*cucharada) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
    elif gr[1] == 'pellizco':
        if gr[0] == '½':
            x = (0.5*pellizco) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        else:
            x = (int(gr[0])*pellizco) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
    elif gr[1] == 'pellizcos':
        if gr[0] == '½':
            x = (0.5*pellizco) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        else:
            x = (int(gr[0])*pellizco) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
    elif gr[1] == 'diente':
        if gr[0] == '½':
            x = (0.5*diente) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        else:
            x = (int(gr[0])*diente) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
    elif gr[1] == 'dientes':
        if gr[0] == '½':
            x = (0.5*diente) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        else:
            x = (int(gr[0])*diente) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()    
    elif gr[1] == 'pastilla':
        if df_clean[df_clean.product_id==prod_id].reference_unit.squeeze() == 'ud':
            if gr[0] == '½':
                x = 0.5 * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
            else:
                x = int(gr[0]) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        else:
            x = (int(gr[0])*pastilla) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
    elif gr[1] == 'pastillas':
        if df_clean[df_clean.product_id==prod_id].reference_unit.squeeze() == 'ud':
            x = int(gr[0]) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        else:
            x = (int(gr[0])*pastilla) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
    elif gr[1] == 'ramita':
        if df_clean[df_clean.product_id==prod_id].reference_unit.squeeze() == '100g':
            if gr[0] == '½':
                x = 0.5 * ramita_perejil * 10 * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
            else:
                x = (int(gr[0])*ramita_perejil)* 10 * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        else:
            x = (int(gr[0])*ramita_perejil) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
    elif gr[1] == 'ramitas':
        if df_clean[df_clean.product_id==prod_id].reference_unit.squeeze() == '100g':
            if gr[0] == '½':
                x = 0.5 * ramita_perejil * 10 * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
            else:
                x = (int(gr[0])*ramita_perejil)* 10 * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        else:
            x = (int(gr[0])*ramita_perejil) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
    elif gr[1] == '-':
        if gr[3] == 'g':
            x = (int(gr[0])/1000) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        elif gr[3] == 'kg':
            x = int(gr[0]) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        elif gr[3] == 'l':
            x = int(gr[0]) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        elif gr[3] == 'hebra':
            x = (int(gr[0]) * hebra) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        elif gr[3] == 'hebras':
            x = (int(gr[0]) * hebra) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        elif gr[3] == 'cucharadita':
            if gr[0] == '½':
                x = (0.5*cucharadita) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
            else:
                x = (int(gr[0])*cucharadita) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        elif gr[3] == 'cucharaditas':
            if gr[0] == '½':
                x = (0.5*cucharadita) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
            else:
                x = (int(gr[0])*cucharadita) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()    
        elif gr[3] == 'cucharada':
            if gr[0] == '½':
                x = (0.5*cucharada) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
            else:
                x = (int(gr[0])*cucharada) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        elif gr[3] == 'cucharadas':
            if gr[0] == '½':
                x = (0.5*cucharada) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
            else:
                x = (int(gr[0])*cucharada) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()    
        elif gr[3] == 'pellizco':
            if gr[0] == '½':
                x = (0.5*pellizco) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
            else:
                x = (int(gr[0])*pellizco) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        elif gr[3] == 'pellizcos':
            if gr[0] == '½':
                x = (0.5*pellizco) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
            else:
                x = (int(gr[0])*pellizco) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()    
        elif gr[3] == 'diente':
            if gr[0] == '½':
                x = (0.5*diente) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
            else:
                x = (int(gr[0])*diente) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        elif gr[3] == 'dientes':
            if gr[0] == '½':
                x = (0.5*diente) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
            else:
                x = (int(gr[0])*diente) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()    
        elif gr[3] == 'pastilla':
            if df_clean[df_clean.product_id==prod_id].reference_unit.squeeze() == 'ud':
                if gr[0] == '½':
                    x = 0.5 * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
                else:
                    x = int(gr[0]) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
            else:
                x = (int(gr[0])*pastilla) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        elif gr[3] == 'pastillas':
            if df_clean[df_clean.product_id==prod_id].reference_unit.squeeze() == 'ud':
                x = int(gr[0]) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
            else:
                x = (int(gr[0])*pastilla) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        elif gr[3] == 'ramita':
            if df_clean[df_clean.product_id==prod_id].reference_unit.squeeze() == '100g':
                if gr[0] == '½':
                    x = 0.5 * ramita_perejil * 10 * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
                else:
                    x = (int(gr[0])*ramita_perejil) * 10 * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
            else:
                x = (int(gr[0])*ramita_perejil) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        elif gr[3] == 'ramitas':
            if df_clean[df_clean.product_id==prod_id].reference_unit.squeeze() == '100g':
                if gr[0] == '½':
                    x = 0.5 * ramita_perejil * 10 * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
                else:
                    x = (int(gr[0])*ramita_perejil)* 10 * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
            else:
                x = (int(gr[0])*ramita_perejil) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
    elif gr[1] == '½':
        if gr[2] == 'cucharadita':
            x = (int(gr[0]) * 0.5 * cucharadita) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        elif gr[2] == 'cucharaditas':
            x = (int(gr[0]) * 0.5 * cucharadita) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        elif gr[2] == 'cucharada':
            x = (int(gr[0]) * 0.5 * cucharada) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        elif gr[2] == 'cucharadas':
            x = (int(gr[0]) * 0.5 * cucharada) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        elif gr[2] == 'diente':
            x = (int(gr[0]) * 0.5 * diente) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        elif gr[2] == 'dientes':
            x = (int(gr[0]) * 0.5 * diente) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        elif gr[2] == 'pastilla':
            if df_clean[df_clean.product_id==prod_id].reference_unit.squeeze() == 'ud':
                x = int(gr[0]) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
            else:
                x = (int(gr[0]) * 0.5 * pastilla) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        elif gr[2] == 'pastillas':
            if df_clean[df_clean.product_id==prod_id].reference_unit.squeeze() == 'ud':
                x = int(gr[0]) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
            else:
                x = (int(gr[0]) * 0.5 * pastilla) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
    else:
        if gr[1] == 'bien':
            x = 0
        elif df_clean[df_clean.product_id==prod_id].reference_unit.squeeze() == 'ud':
            x = int(gr[0]) * df_clean[df_clean.product_id==prod_id].reference_price.squeeze()
        elif df_clean[df_clean.product_id==prod_id].reference_unit.squeeze() == 'docena':
            x = int(gr[0]) * (df_clean[df_clean.product_id==prod_id].reference_price.squeeze() / 12)
        else:
            x = 0
    return(round(x, 2))

In [20]:
for i in df_recetas[df_recetas.nombre_receta=='arroz caldoso con especias'].ingredientes.squeeze():
    prod_id = get_key_generico(i, a_dia)[0]
    print(precio_gramaje(i, df, prod_id))

In [21]:
def get_recipe_prices(recipe_list, supermarket_list, nom_supermarkets, df_clean):
    count = 0
    for i, h in zip (supermarket_list, nom_supermarkets):
        product_id = []
        match = []
        probabilities = []
        price = []
        exact_price = []
        name = []
    
    #for i in supermarket_list:
        for j in recipe_list:
            if get_key_generico(j, i)[0] != 'No product_ID':
                prod_id = get_key_generico(j, i)[0]
                prod_match = get_key_generico(j, i)[1]
                prod_perc = get_key_generico(j, i)[2]
                
                product_id.append(prod_id)
                match.append(prod_match)
                probabilities.append(prod_perc)
                price.append(df_clean[df_clean.product_id==prod_id].price.squeeze())
                name.append(df_clean[df_clean.product_id==prod_id].name.iloc[0])
     
    
    # buscamos el precio exacto en base al gramaje    
                exact_price.append(round(precio_gramaje(j, df_clean, prod_id), 2))
        
            else:
                product_id.append(get_key_generico(j, i)[0])
                match.append(get_key_generico(j, i)[1])
                probabilities.append(get_key_generico(j, i)[2])
                price.append('NaN')
                name.append('NaN')
                exact_price.append('NaN')
        
        
        # creamos el dataframe con todos los ingredientes de la receta    
        df = pd.DataFrame({
            'receta': recipe_list,
            #'potential matches': match,
            #'product_id': product_id,
            #'probability of match': probabilities,
            f'{h}_products': name,
            f'{h}_price_supermarket': price,
            f'{h}_cost_ingredients_used': exact_price})
        df[f'{h}_price_supermarket'] = df[f'{h}_price_supermarket'].astype(float)
        df[f'{h}_cost_ingredients_used'] = df[f'{h}_cost_ingredients_used'].astype(float)
        
        if count > 0:
            df_merge = pd.merge(df_merge, df, on=['receta'])
        else:
            df_merge = df
            count += 1
        
    return df_merge

In [22]:
df_recetas['nombre_receta'].unique()  

array(['Albóndigas en salsa española', 'Arroz caldoso con especias',
       'Arroz tres delicias',
       'Espaguetis integrales con pollo y verduras',
       'Fideuá negra de calamares',
       'Garbanzos con tomate, piquillos y atún',
       'Garbanzos en salsa verde con salmón',
       'Lasaña de pollo y espinacas', 'Lentejas estofadas',
       'Verduras y pavo al vapor'], dtype=object)

In [23]:
df_merge = get_recipe_prices(df_recetas[df_recetas.nombre_receta=='Albóndigas en salsa española'].ingredientes.squeeze(), 
                             tablas_intermedias, nombres_supermercados, df)
df_merge

,receta,mercadona_products,mercadona_price_supermarket,mercadona_cost_ingredients_used,dia_products,dia_price_supermarket,dia_cost_ingredients_used,carrefour_products,carrefour_price_supermarket,carrefour_cost_ingredients_used
0,200 g de zanahoria troceada,Zanahorias,0.69,0.28,Zanahoria bolsa 1 Kg,0.89,0.18,Zanahoria Carrefour 1 kg,0.85,0.17
1,2 dientes de ajo,Ajos morados,1.45,0.02,Ajo morado malla 250 gr,1.49,0.02,Ajo morado ristra 500 g,3.85,0.03
2,2 dientes de ajo,Ajos morados,1.45,0.02,Ajo morado malla 250 gr,1.49,0.02,Ajo morado ristra 500 g,3.85,0.03
3,2 dientes de ajo,Ajos morados,1.45,0.02,Ajo morado malla 250 gr,1.49,0.02,Ajo morado ristra 500 g,3.85,0.03
4,2 dientes de ajo,Ajos morados,1.45,0.02,Ajo morado malla 250 gr,1.49,0.02,Ajo morado ristra 500 g,3.85,0.03
5,2 dientes de ajo,Ajos morados,1.45,0.02,Ajo morado malla 250 gr,1.49,0.02,Ajo morado ristra 500 g,3.85,0.03
6,2 dientes de ajo,Ajos morados,1.45,0.02,Ajo morado malla 250 gr,1.49,0.02,Ajo morado ristra 500 g,3.85,0.03
7,2 dientes de ajo,Ajos morados,1.45,0.02,Ajo morado malla 250 gr,1.49,0.02,Ajo morado ristra 500 g,3.85,0.03
8,2 dientes de ajo,Ajos morados,1.45,0.02,Ajo morado malla 250 gr,1.49,0.02,Ajo morado ristra 500 g,3.85,0.03
9,1 - 2 ramitas de perejil fresco (sólo las hojas),Perejil troceado,0.70,0.02,Perejil manojo 50 gr,0.89,0.02,Perejil manojo 1 ud,0.89,0.00


In [24]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38 entries, 0 to 37
Data columns (total 10 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   receta                           38 non-null     object 
 1   mercadona_products               38 non-null     object 
 2   mercadona_price_supermarket      38 non-null     float64
 3   mercadona_cost_ingredients_used  38 non-null     float64
 4   dia_products                     38 non-null     object 
 5   dia_price_supermarket            38 non-null     float64
 6   dia_cost_ingredients_used        38 non-null     float64
 7   carrefour_products               38 non-null     object 
 8   carrefour_price_supermarket      38 non-null     float64
 9   carrefour_cost_ingredients_used  38 non-null     float64
dtypes: float64(6), object(4)
memory usage: 3.3+ KB


In [25]:
def resultados(df, lista_supermercados):
    count = 0
    for i in lista_supermercados:
        resultado = [round(df[f'{i}_cost_ingredients_used'].sum(), 2), 
                     round(df[f'{i}_price_supermarket'].sum(), 2)]
    
    # creamos el dataframe con todos los ingredientes de la receta
        df_pre = pd.DataFrame({f'{i}': resultado}, index = ['Coste Ingredientes Usados', 'Coste Cesta de la Compra'])
        if count > 0:
            df_prices = pd.concat([df_prices, df_pre], axis=1)
        else:
            df_prices = df_pre
            count += 1

    return df_prices

In [26]:
m_t = resultados(df_merge, nombres_supermercados)
m_t

,mercadona,dia,carrefour
Coste Ingredientes Usados,8.92,11.28,9.43
Coste Cesta de la Compra,58.40,63.31,76.56


In [27]:
h = print(m_t.loc['Coste Ingredientes Usados']['mercadona'], '€')
h

8.92 €


In [28]:
print('El precio de esta receta es de:\n', round(df_merge['mercadona_cost_ingredients_used'].sum(), 2), 
      '€ en mercadona\n', round(df_merge['dia_cost_ingredients_used'].sum(), 2), '€ en dia\n', 
      round(df_merge['carrefour_cost_ingredients_used'].sum(), 2), '€ en carrefour\n\n')
print('El precio de la cesta de la compra de esta receta es de:\n', 
      round(df_merge['mercadona_price_supermarket'].sum(), 2), '€ en mercadona\n',
      round(df_merge['dia_price_supermarket'].sum(), 2), '€ en dia\n',
      round(df_merge['carrefour_price_supermarket'].sum(), 2), '€ en carrefour\n')

El precio de esta receta es de:
 8.92 € en mercadona
 11.28 € en dia
 9.43 € en carrefour


El precio de la cesta de la compra de esta receta es de:
 58.4 € en mercadona
 63.31 € en dia
 76.56 € en carrefour



# Cálculo de todos los precios en Df_recetas

In [29]:
def price_mercadona (df):
    return df.loc['Coste Ingredientes Usados']['mercadona']

In [30]:
# Actualizamos el df original de las recetas de cocina con los precios

df_recetas['px_receta_mercadona'] = df_recetas.apply(lambda x: price_mercadona(resultados(get_recipe_prices(x.ingredientes, 
                             tablas_intermedias, nombres_supermercados, df), nombres_supermercados)), axis=1)
df_recetas['px_ración_mercadona'] = round(df_recetas['px_receta_mercadona'] / df_recetas['numero_raciones'],2)
df_recetas

,nombre_receta,ingredientes,tiempo_dedicado_preparación_min,tiempo_cocinado_robot_min,numero_raciones,kcal_por_racion,px_receta_mercadona,px_ración_mercadona
9,Albóndigas en salsa española,"[200 g de zanahoria troceada, 2 dientes de ajo...",10,55,6,480,8.92,1.49
6,Arroz caldoso con especias,"[4 dientes de ajo, 100 g de cebolla, 30 - 50 g...",15,40,4,321,5.69,1.42
2,Arroz tres delicias,[100 - 150 g de jamón cocido en trozos gruesos...,15,40,6,346,4.52,0.75
3,Espaguetis integrales con pollo y verduras,"[200 g de cebolla en cuartos, 1 diente de ajo,...",15,25,6,359,5.17,0.86
7,Fideuá negra de calamares,"[150 g de cebolla cortada en cuartos, 2 diente...",15,45,4,496,9.61,2.40
1,"Garbanzos con tomate, piquillos y atún","[2 dientes de ajo, 300 g de cebolla en trozos,...",10,45,6,247,5.21,0.87
4,Garbanzos en salsa verde con salmón,[10 ramitas de perejil fresco limpio y seco (s...,10,25,4,488,12.30,3.08
8,Lasaña de pollo y espinacas,"[9 placas de lasaña secas precocidas, 200 g de...",10,60,8,491,12.66,1.58
0,Lentejas estofadas,"[350 g de lentejas castellanas secas, 50 g de ...",10,45,6,270,2.25,0.38
5,Verduras y pavo al vapor,"[500 g de agua (para el vapor), 120 g de zanah...",10,30,4,341,6.80,1.70


In [31]:
def price_dia (df):
    return df.loc['Coste Ingredientes Usados']['dia']

In [32]:
df_recetas['px_receta_dia'] = df_recetas.apply(lambda x: price_dia(resultados(get_recipe_prices(x.ingredientes, 
                             tablas_intermedias, nombres_supermercados, df), nombres_supermercados)), axis=1)
df_recetas['px_ración_dia'] = round(df_recetas['px_receta_dia'] / df_recetas['numero_raciones'],2)
#df_recetas

In [33]:
def price_carrefour (df):
    return df.loc['Coste Ingredientes Usados']['carrefour']

In [34]:
df_recetas['px_receta_carrefour'] = df_recetas.apply(lambda x: price_carrefour(resultados(get_recipe_prices(x.ingredientes, 
                             tablas_intermedias, nombres_supermercados, df), nombres_supermercados)), axis=1)
df_recetas['px_ración_carrefour'] = round(df_recetas['px_receta_carrefour'] / df_recetas['numero_raciones'],2)
df_recetas

,nombre_receta,ingredientes,tiempo_dedicado_preparación_min,tiempo_cocinado_robot_min,numero_raciones,kcal_por_racion,px_receta_mercadona,px_ración_mercadona,px_receta_dia,px_ración_dia,px_receta_carrefour,px_ración_carrefour
9,Albóndigas en salsa española,"[200 g de zanahoria troceada, 2 dientes de ajo...",10,55,6,480,8.92,1.49,11.28,1.88,9.43,1.57
6,Arroz caldoso con especias,"[4 dientes de ajo, 100 g de cebolla, 30 - 50 g...",15,40,4,321,5.69,1.42,5.48,1.37,5.41,1.35
2,Arroz tres delicias,[100 - 150 g de jamón cocido en trozos gruesos...,15,40,6,346,4.52,0.75,4.65,0.78,5.10,0.85
3,Espaguetis integrales con pollo y verduras,"[200 g de cebolla en cuartos, 1 diente de ajo,...",15,25,6,359,5.17,0.86,6.08,1.01,5.14,0.86
7,Fideuá negra de calamares,"[150 g de cebolla cortada en cuartos, 2 diente...",15,45,4,496,9.61,2.40,11.14,2.78,7.99,2.00
1,"Garbanzos con tomate, piquillos y atún","[2 dientes de ajo, 300 g de cebolla en trozos,...",10,45,6,247,5.21,0.87,5.66,0.94,5.53,0.92
4,Garbanzos en salsa verde con salmón,[10 ramitas de perejil fresco limpio y seco (s...,10,25,4,488,12.30,3.08,12.01,3.00,14.78,3.70
8,Lasaña de pollo y espinacas,"[9 placas de lasaña secas precocidas, 200 g de...",10,60,8,491,12.66,1.58,16.73,2.09,15.10,1.89
0,Lentejas estofadas,"[350 g de lentejas castellanas secas, 50 g de ...",10,45,6,270,2.25,0.38,2.67,0.44,2.35,0.39
5,Verduras y pavo al vapor,"[500 g de agua (para el vapor), 120 g de zanah...",10,30,4,341,6.80,1.70,7.42,1.86,6.50,1.62


In [35]:
df_recetas.to_csv(route + 'df_recetas_con_precios.csv')

In [36]:
df_recetas.to_csv(route_2 + 'df_recetas_con_precios.csv')